In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
from glob import glob
from PIL import Image

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# Hyperparameter
BATCH_SIZE = 64
LR = 1e-3        # 논문에서는 1e-2
EPOCHS = 20      # 논문에서는 74

In [4]:
stl_train = torchvision.datasets.STL10(root='./stl_train/',
                                       split="train",
                                       download=True)

stl_test = torchvision.datasets.STL10(root='./stl_test/',
                                      split='test',
                                      download=True)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./stl_train/stl10_binary.tar.gz to ./stl_train/


  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./stl_test/stl10_binary.tar.gz to ./stl_test/


In [14]:
trainLoader = torch.utils.data.DataLoader(stl_train,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

valLoader = torch.utils.data.DataLoader(stl_test,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False)

In [15]:
train_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((256, 256)),
                                      transforms.RandomCrop((224, 224)),
                                      transforms.RandomHorizontalFlip()
                                     ])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((224, 224)),
                                    ])

In [16]:
stl_train.transform = train_transform
stl_test.transform = test_transform

In [17]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()

        self.conv_block = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2),
                                        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2),
                                        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2),
                                        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(512, 512, kernel_size=1, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2),
                                        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.Conv2d(512, 512, kernel_size=1, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2)
                                        )
        
        self.classifier = nn.Sequential(nn.Linear(32768, 4096),
                                        nn.Dropout(p=0.5),
                                        nn.Linear(4096, 4096),
                                        nn.Dropout(p=0.5),
                                        nn.Linear(4096, 10)
                                        )
        
        for block in [self.conv_block, self.classifier]:
            for layer in block:
                if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                    nn.init.normal_(layer.weight, mean=0, std=0.1)
        

    def forward(self, x):
        x = self.conv_block(x)
        x = x.flatten(start_dim=1)
        out = self.classifier(x)
        return out

In [18]:
def train_loop(model, Loader):
    train_loss = 0
    train_acc = 0
    model.train()
    for data, target in tqdm(Loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (torch.argmax(output, dim=1) == target).sum().item()
    return train_loss / len(Loader), train_acc / len(Loader.dataset)

In [19]:
def val_loop(model, Loader):
    val_loss = 0
    val_acc = 0
    model.eval()
    with torch.no_grad():
        for data, target in tqdm(Loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target)

            val_loss += loss.item()
            val_acc += (torch.argmax(output, dim=1) == target).sum().item()
    return val_loss / len(Loader), val_acc / len(Loader.dataset)

In [20]:
model = VGG16().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
 # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [21]:
summary(model, (3, 224, 224), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [22]:
for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = train_loop(model, trainLoader)
    val_loss, val_acc = val_loop(model, valLoader)

    print(f'\n[[  Epoch {epoch:2d} / {EPOCHS}  ]]\n')
    print(f'Train | Loss {train_loss:.4f}, Accuracy : {train_acc*100:.2f} %')
    print(f'Valid | Loss {val_loss:.4f}, Accuracy : {val_acc*100:.2f} %\n')

    # scheduler.step()

100%|██████████| 125/125 [01:24<00:00,  1.48it/s]


[[  Epoch  1 / 20  ]]

Train | Loss 2076942.9813, Accuracy : 15.16 %
Valid | Loss 1939.9749, Accuracy : 23.40 %



100%|██████████| 125/125 [01:24<00:00,  1.48it/s]


[[  Epoch  2 / 20  ]]

Train | Loss 1958.2627, Accuracy : 19.12 %
Valid | Loss 553.2977, Accuracy : 23.15 %



100%|██████████| 125/125 [01:24<00:00,  1.48it/s]


...

[[  Epoch 18 / 20  ]]

Train | Loss 2.6084, Accuracy : 9.62 %
Valid | Loss 2.5391, Accuracy : 10.05 %



100%|██████████| 125/125 [01:24<00:00,  1.48it/s]


[[  Epoch 19 / 20  ]]

Train | Loss 2.6605, Accuracy : 10.26 %
Valid | Loss 2.5116, Accuracy : 10.05 %



100%|██████████| 125/125 [01:24<00:00,  1.49it/s]

[[  Epoch 20 / 20  ]]

Train | Loss 2.5957, Accuracy : 9.70 %
Valid | Loss 2.5049, Accuracy : 9.93 %

